In [2]:
import hashlib

import numpy as np

from helper_loader import *
from histalign.backend.registration.alignment import (
    INTERPOLATED_VOLUMES_CACHE_DIRECTORY,
    _module_logger,
    build_alignment_volume,
)

set_log_level("DEBUG")

In [3]:
client = Client(n_workers=4, threads_per_worker=1, memory_limit="2GB")
client

In [5]:
alignment_path = Path(
    "/home/ediun/git/histalign/projects/project_cortical_depth/93e6cae680"
)

array = build_alignment_volume(alignment_path, return_raw_array=True)

FileNotFoundError: [Errno 2] No such file or directory: '/home/ediun/git/histalign/projects/project_cortical_depth/93e6cae680'

In [4]:
# def generate_hash_from_targets(targets: list[Path]) -> str:
#     return hashlib.md5("".join(map(str, targets)).encode("UTF-8")).hexdigest()
#
#
# def interpolate_sparse_3d_array(
#     array: np.ndarray,
#     reference_mask: Optional[np.ndarray] = None,
#     pre_masked: bool = False,
#     kernel: str = "multiquadric",
#     neighbours: int = 27,
#     epsilon: int = 1,
#     degree: Optional[int] = None,
#     chunk_size: Optional[int] = 1_000_000,
#     recursive: bool = False,
#     use_cache: bool = False,
#     alignment_directory: str | Path = "",
#     mask_name: str = "",
# ) -> np.ndarray:
#     start_time = time.perf_counter()
#
#     if use_cache and not alignment_directory:
#         raise ValueError(
#             "Cannot use cache without 'alignment_directory' identifying information."
#         )
#     if use_cache and reference_mask is not None and not mask_name:
#         raise ValueError(
#             "Cannot use cache with reference mask but no 'mask_name' "
#             "identifying information."
#         )
#     if isinstance(alignment_directory, str):
#         alignment_directory = Path(alignment_directory)
#
#     if reference_mask is not None and (array_shape := array.shape) != (
#         reference_shape := reference_mask.shape
#     ):
#         raise ValueError(
#             f"Array and reference mask have different shapes "
#             f"({array_shape} vs {reference_shape})."
#         )
#
#     # Mask the array if necessary
#     if reference_mask is not None and not pre_masked:
#         array = np.where(reference_mask, array, 0)
#
#     cache_hash = generate_hash_from_targets(gather_alignment_paths(alignment_directory))
#     mask_name = "-".join(mask_name.split(" ")).lower()
#     cache_path = (
#         INTERPOLATED_VOLUMES_CACHE_DIRECTORY
#         / f"{cache_hash}{f'_{mask_name}' if reference_mask is not None else ''}.npz"
#     )
#     if cache_path.exists() and use_cache:
#         _module_logger.debug("Found cached array. Loading from file.")
#
#         return np.load(cache_path)["array"]
#
#     interpolated_array = array.copy()
#     interpolated_array = interpolated_array.astype(np.float64)
#
#     if reference_mask is None:
#         # Interpolate the whole grid
#         target_coordinates = tuple(
#             array.flatten().astype(int)
#             for array in np.meshgrid(
#                 np.linspace(
#                     0, interpolated_array.shape[0] - 1, interpolated_array.shape[0]
#                 ),
#                 np.linspace(
#                     0, interpolated_array.shape[1] - 1, interpolated_array.shape[1]
#                 ),
#                 np.linspace(
#                     0, interpolated_array.shape[2] - 1, interpolated_array.shape[2]
#                 ),
#                 indexing="ij",
#             )
#         )
#     else:
#         # Interpolate only non-zero coordinates of mask
#         target_coordinates = np.nonzero(reference_mask)
#     target_points = np.array(target_coordinates).T
#
#     if chunk_size is None:
#         chunk_size = target_points.shape[0]
#
#     _module_logger.info(
#         f"Starting interpolation with parameters "
#         f"{{"
#         f"kernel: {kernel}, "
#         f"neighbours: {neighbours}, "
#         f"epsilon: {epsilon}, "
#         f"degree: {degree}, "
#         f"chunk size: {chunk_size:,}, "
#         f"recursive: {recursive}"
#         f"}}."
#     )
#
#     failed_chunks = []
#     previous_target_size = target_points.shape[0]
#     while True:
#         known_coordinates = np.nonzero(interpolated_array)
#         known_points = np.array(known_coordinates).T
#
#         known_values = array[known_coordinates]
#
#         interpolator = RBFInterpolator(
#             known_points,
#             known_values,
#             kernel=kernel,
#             neighbors=neighbours,
#             epsilon=epsilon,
#             degree=degree,
#         )
#
#         def interpolation_function(chunk: np.ndarray) -> np.ndarray:
#             try:
#                 interpolated_data = interpolator(chunk)
#             except np.linalg.LinAlgError:
#                 interpolated_data = np.zeros(shape=(chunk.shape[0],), dtype=np.float64)
#
#             return interpolated_data
#
#         chunk_start = 0
#         chunk_end = chunk_size
#         chunk_index = 1
#         chunk_count = math.ceil(target_points.shape[0] / chunk_size)
#         while chunk_start < target_points.shape[0]:
#             _module_logger.info(
#                 f"Interpolating chunk {chunk_index}/{chunk_count} "
#                 f"({chunk_index / chunk_count:.0%})."
#             )
#
#             chunk_coordinates = tuple(
#                 coordinate[chunk_start:chunk_end] for coordinate in target_coordinates
#             )
#             chunk_points = target_points[chunk_start:chunk_end]
#             chunk_points = da.from_array(chunk_points)
#
#             try:
#                 interpolated_array[chunk_coordinates] = interpolator(chunk_points)
#             except np.linalg.LinAlgError:
#                 failed_chunks.append([chunk_start, chunk_end])
#                 _module_logger.info(f"Failed to interpolate chunk {chunk_index}.")
#
#             chunk_start += chunk_size
#             chunk_end += chunk_size
#             chunk_index += 1
#
#         if not recursive or len(failed_chunks) == 0:
#             break
#
#         # Prepare the next loop
#         target_coordinates = tuple(
#             np.concatenate(
#                 [target_coordinate[start:end] for start, end in failed_chunks]
#             )
#             for target_coordinate in target_coordinates
#         )
#         target_points = np.array(target_coordinates).T
#         failed_chunks = []
#
#         # Avoid infinitely looping
#         if previous_target_size == target_points.shape[0]:
#             _module_logger.error(
#                 f"Interpolation is not fully solvable with current combination of "
#                 f"kernel, neighbours parameter and chunk size. "
#                 f"Returning current result."
#             )
#             break
#         previous_target_size = target_points.shape[0]
#
#         _module_logger.info(
#             f"There were {len(failed_chunks)} failed chunks of size {chunk_size}. "
#             f"Recursing with newly interpolated data."
#         )
#
#     total_time = time.perf_counter() - start_time
#     total_hours, remaining_time = divmod(total_time, 3600)
#     total_minutes, total_seconds = divmod(remaining_time, 60)
#     time_string = (
#         f"{f'{total_hours:.0f}h' if total_hours else ''}"
#         f"{f'{total_minutes:>2.0f}m' if total_minutes else ''}"
#         f"{total_seconds:>2.0f}s"
#     )
#     _module_logger.info(f"Finished interpolation in {time_string}.")
#
#     if use_cache:
#         _module_logger.debug("Caching interpolated array to file.")
#         os.makedirs(INTERPOLATED_VOLUMES_CACHE_DIRECTORY, exist_ok=True)
#         np.savez_compressed(cache_path, array=interpolated_array)
#
#     return interpolated_array

In [5]:
def get_tuple_all_coordinates(array_shape: tuple[int, ...]) -> tuple[np.ndarray, ...]:
    assert max(array_shape) < 2**16 - 1, "Supported coordinate dtype is np.uint16"

    return tuple(
        array.flatten().astype(int)
        for array in np.meshgrid(
            *[
                np.linspace(0, array_shape[i] - 1, array_shape[i], dtype=np.uint16)
                for i in range(len(array_shape))
            ],
            indexing="ij",
        )
    )

In [6]:
def interpolate_dask(array: np.ndarray) -> np.ndarray:
    interpolated_array = array.copy().astype(np.float64)

    known_coordinates = np.nonzero(interpolated_array)
    known_points = np.array(known_coordinates).T
    known_points = da.from_array(known_points)

    known_values = array[known_coordinates]
    known_values = da.from_array(known_values)

    interpolator = RBFInterpolator(
        known_points,
        known_values,
        kernel="multiquadric",
        neighbors=16,
        epsilon=1,
        degree=None,
    )

    def interpolation_function(chunk: np.ndarray) -> np.ndarray:
        try:
            interpolated_data = interpolator(chunk)
        except np.linalg.LinAlgError:
            interpolated_data = np.zeros(shape=(chunk.shape[0],), dtype=np.float64)

        return interpolated_data

    target_coordinates = get_tuple_all_coordinates(interpolated_array.shape)
    target_points = np.array(target_coordinates, np.uint16).T
    target_points = da.from_array(target_points)

    return da.map_blocks(
        interpolation_function, target_points, drop_axis=1, dtype=np.float64
    )

In [7]:
interpolated_data = interpolate_dask(array)
interpolated_data.compute()

/home/ediun/.local/share/hatch/env/virtual/histalign/oFfvMJyE/histalign/lib/python3.10/site-packages/distributed/client.py:3361: UserWarning: Sending large graph of size 461.07 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
2024-11-07 12:08:42,437 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 1.32 GiB -- Worker memory limit: 1.86 GiB
2024-11-07 12:08:58,885 - distributed.worker.memory - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memory: 1.49 GiB -- Worker memory limit: 1.86 GiB
2024-11-07

KilledWorker: Attempted to run task ('interpolation_function-624c9e9c5b974a47ea5cd0aa38afe857', 0) on 4 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://127.0.0.1:39071. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.